# Deaths by age group and federal states in Germany from 2016 to 2021
> Interactive analysis of death rates before and during Covid19.

- toc:true
- badges: true
- comments: true
- author: Johannes Tomasoni
- image: images/mortality_germany.png
- categories: [EDA]

[:de:](https://joatom.github.io/ai_curious/eda/2021/04/06/mortality-germany_de.html) [:us:](https://joatom.github.io/ai_curious/eda/2021/04/06/mortality-germany.html) 


In [ ]:
#hide

# uncomment for use in BINDER or COLAB

#!pip install xlrd
#!pip install openpyxl
#!pip install altair
#!pip install pandas

In [2]:
#hide

import pandas as pd
import altair as alt
from IPython.display import display, HTML
from datetime import date

import warnings
warnings.filterwarnings("ignore")

In this blog post, I want to find out how easy it is to create interactive charts in notebooks. As a framework, I will use [Altair](https://altair-viz.github.io/), as it can be easily integrated into my blog [[FP20](https://fastpages.fast.ai/jupyter/2020/02/20/test.html)].

In order to make the contribution interesting, I evaluate the death rates from Germany over the last five years. I construct two periods - the first period before and the second one during Covid19.

We will analyze these questions: 
- How do deaths per age group and month change?
- How do deaths per federal state change?

The first part describes the origin and processing of the data. Then, charts are built around the above questions.
Finally, a technical conclusion to the framework follows.

> Important: The death rates are not an indicator of the danger of Covid19. They only reflect the number of people who died retrospectively.

# Source of data
For the analysis of deaths in Germany, the current death data from the *Statistisches Bundesamt* (Federal Statistical Office) [SB21] are used. The data include the number of deaths by age group or federal state on a monthly basis. In this blog post, the monthly death figures for the period March 2016 to February 2021 are examined. The most recent data are currently only available until February 2021 and include some estimations, which are explained in the data source in the tab "Hinweise".
In addition to the death figures, data on the population density of the federal states are processed [SB20]. These figures refere to the reference date 31.12.2019.

## Procession of the data 
## Preprocessing
The evaluation period is limited to March 2016 until February 2021. The period in which Corona was heavily active in Germany is simplified here to March 2020 (when Covid19 triggered the first major social changes in Germany) until February 2021 (orange). The pre-Covid19 period is set to March 2016 until February 2020 (blue). Thus, the Covid19 period covers exactly one year and the pre-Covid19 period exactly four years. Thus, both periods remain comparable without serious seasonal deviations. The split of the periods is illustrated in the diagram below.

In [3]:
#hide

# Translations and mapping dictionaries

LANG = 'en' # 'de' 'en'

if LANG == 'en':
    MONTHS = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    # titles / labels 
    T_AGE = 'Age'
    T_AGE_GRP = 'Age group'
    T_ALL = 'All'
    T_AND = 'and'
    T_DEATHS = 'Deaths'
    T_DEATHS_BC = 'Deaths before Covid19 (monthly average)'
    T_DEATHS_BY_AGE = 'Deaths by Age (monthly average)'
    T_DEATHS_BY_STATE = 'Deaths by State (monthly average)'
    T_DEATHS_C = 'Deaths during Covid19 (monthly average)'
    T_DEATHS_MONTHLY = 'Deaths (monthly average)'
    T_DEATHS_HTH = 'Deaths per 100,000 (monthly average)'
    T_DENSITY = 'Population per km\xb2'
    T_DENSITY2 = 'Population density'
    T_MEASURE = 'Measure'
    T_MONTH = 'Month'
    T_MORT_INC = 'Increased mortality'
    T_PERIOD = 'Period'
    T_PERIOD_SPLIT = 'Split of the Periods'
    T_PER_HTH = 'per 100,000'
    T_STATE = 'State'
    T_STATE_MEASURES = ['Deaths', 'Deaths per 100,000']
    
elif LANG == 'de':
    MONTHS = ['Jan', 'Feb', 'Mrz', 'Apr', 'Mai', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez']
    
    # titels / labels
    T_AGE = 'Alter'
    T_AGE_GRP = 'Altersgruppe'
    T_ALL = 'Alle'
    T_AND = 'und'
    T_DEATHS = 'Todesfälle'
    T_DEATHS_BC = 'Tote vor Covid19 (monatl. \xd8)'
    T_DEATHS_BY_AGE = 'Todesfälle nach Alter (monatl. \xd8)'
    T_DEATHS_BY_STATE = 'Todesfälle nach Bundesland (monatl. \xd8)'
    T_DEATHS_C = 'Tote seit Covid19 (monatl. \xd8)'
    T_DEATHS_MONTHLY = 'Todesfälle (monatl. \xd8)'
    T_DEATHS_HTH = 'Tote je 100.000 Einwohner (monatl. \xd8)'
    T_DENSITY = 'Einwohner je km\xb2'
    T_DENSITY2 = 'Bevölkerungsdichte'
    T_MEASURE = 'Kennzahl'
    T_MONTH = 'Monat'
    T_MORT_INC = 'Zunahme der Todesfälle'
    T_PERIOD = 'Zeitraum'
    T_PERIOD_SPLIT = 'Aufteilung der Zeiträume'
    T_PER_HTH = 'je 100.000'
    T_STATE = 'Bundesland'
    T_STATE_MEASURES = ['Todesfälle', 'Tote je 100.000']

AGE_MAP = {'0-15' : 15, 
           '15-30' : 30, 
           '30-35' : 35, 
           '35-40' : 40, 
           '40-45' : 45, 
           '45-50' : 50, 
           '50-55' : 55, 
           '55-60' : 60,
           '60-65' : 65, 
           '65-70' : 70, 
           '70-75' : 75, 
           '75-80' : 80, 
           '80-85' : 85, 
           '85-90' : 90, 
           '90-95' : 95,
           '95+' : 100,
           '0-65' : 65,
           '65+': 100}

STATE_MAP = {'Schleswig-Holstein': 'SH', 
             'Hamburg': 'HH', 
             'Niedersachsen': 'NI', 
             'Bremen': 'HB',
             'Nordrhein-Westfalen': 'NW', 
             'Hessen':'HE', 
             'Rheinland-Pfalz': 'RP',
             'Baden-Württemberg': 'BW', 
             'Bayern': 'BY', 
             'Saarland': 'SL', 
             'Berlin': 'BE', 
             'Brandenburg': 'BB', 
             'Mecklenburg-Vorpommern': 'MV',
             'Sachsen': 'SN',
             'Sachsen-Anhalt': 'ST',
             'Thüringen': 'TH'
}


In [4]:
#hide

data_in = pd.read_excel(io = "https://github.com/joatom/blog-resources/blob/main/mortality_germany/sonderauswertung-sterbefaelle.xlsx?raw=true", sheet_name = 'D_2016-2021_Monate_AG_Ins', header = 8, index_col = 0,engine='openpyxl')

data_in = data_in.drop(['Unnamed: 15', 'Unnamed: 16'], axis = 1).reset_index(drop=True)
data_in.columns = ['year', 'age'] + MONTHS

# drop total
data_in = data_in[data_in.age != 'Insgesamt'].reset_index(drop=True)

# clean age column
data_in['age'] = data_in['age'].str.replace(' u. mehr', '+')
# define upper bound of age range
data_in['age_to'] = data_in['age'].map(AGE_MAP)

#data_in.tail()

In [5]:
#hide

# unpivot on months
data = data_in.melt(id_vars=['year', 'age', 'age_to'] , value_vars = MONTHS).dropna()
data.columns = ['year', 'age', 'age_to', 'month', 'deaths']

# drop January and February 2016 to get four full years
data = data[(data['year']>2016) | (data['month'].isin(MONTHS[2:]))].reset_index(drop=True)

# mark covid period (2020/Mar - 2021/Feb) and non-covid period (2016/Mar - 2020/Feb)
data['period'] = '2020/Mar - 2021/Feb'
data.loc[(data.year < 2020) | ((data.year == 2020) & (data.month.isin(['Jan', 'Feb']))), 'period'] = '2016/Mar - 2020/Feb'

#data.tail()

In [6]:
#hide_input

alt.Chart(data).mark_rect().encode(
    x=alt.X('month:N', sort=MONTHS, title=''),
    y=alt.Y('year:N', title=''),
    color=alt.Color('period', legend=alt.Legend(title=T_PERIOD))
).properties(title = T_PERIOD_SPLIT, width=600, height=200)


alt.Chart(...)

## Aggregation of data
In the calculations, the values are averaged over a period of time. Depending on the evaluation, this happens over the whole period or per month. The first and third quartile of the aggregated data may also be shown as shading in the diagrams.
In some charts, the calculated points are interpolated to increase readability.

> Tip: For some of the charts there are control elements in the upper right corner, such as drop-down boxes. The mouse wheel can be used to zoom and a chart can be reset by double-clicking.

# Deaths by age
The diagram below shows the average number of *deaths per month* per *age group*. One age group covers five years. The point *age 55* includes, for example, all deaths between 50 and 55. Deaths of those over 100 years of age are shown in point *100*.

The values are aggregated over the pre-Covid19 period (blue) and the Covid19 period (orange). In the diagram, the *months* can be selected via drop-down box.

In [7]:
#hide_input

# set html position for dropbox
display(HTML("""
<style>
form.vega-bindings {
  position: absolute;
  right: 50px;
  top: 0px;
}
</style>
"""))


line = alt.Chart(data).mark_line().encode(
    x = alt.X('age_to', title=T_AGE),
    y = alt.Y('mean(deaths)', scale = alt.Scale(domain=[0, 23000])),
    color = alt.Color('period', legend=alt.Legend(title=T_PERIOD)),
)

band = alt.Chart(data).mark_errorband(extent='iqr').encode(
    x = alt.X('age_to', title=T_AGE),
    y = alt.Y('deaths', title=T_DEATHS, scale= alt.Scale(domain=[0, 23000])),
    color = alt.Color('period', legend=alt.Legend(title=T_PERIOD))
)

months_dropdown = alt.binding_select(options=[None]+MONTHS[2:]+MONTHS[:2],labels=[T_ALL]+MONTHS[2:]+MONTHS[:2], name=T_MONTH + ': ')
months_select = alt.selection_single(fields=['month'], bind=months_dropdown, name='month')

filter_line = line.add_selection(
    months_select
)


# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['age_to'], empty='none')
# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(data).mark_point().encode(
    x='age_to:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, alt.Text('mean(deaths):Q', format='.0f'), alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(data).mark_rule(color='gray').encode(
    x='age_to:Q',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
plot_interactive = (
    alt.layer(
        band + filter_line.interactive()  , selectors, points, rules, text
    ).transform_filter(
     months_select 
    ).properties(title=T_DEATHS_BY_AGE, width=800, height=400)
)


def plot_static(month):
    return alt.layer(line + band).transform_filter(alt.datum.month == month).properties(title=month, width=210, height=105)

plot_interactive

alt.LayerChart(...)

Below are the average deaths by age for each month. The list starts with the month of March.

In [8]:
#hide_input

((plot_static(MONTHS[2]) | plot_static(MONTHS[3]) | plot_static(MONTHS[4])) & 
 (plot_static(MONTHS[5]) | plot_static(MONTHS[6]) | plot_static(MONTHS[7])) &
 (plot_static(MONTHS[8]) | plot_static(MONTHS[9]) | plot_static(MONTHS[10])) & 
 (plot_static(MONTHS[11]) | plot_static(MONTHS[0]) | plot_static(MONTHS[1])))


alt.VConcatChart(...)

## Observation
In the age groups under 55, there was no excess mortality during these periods. In the age groups from 80 years and older, mortality increased massively.

# Deaths by state
In the next diagram, the age groups *0-65* and *65+* can be evaluated for the two periods grouped by *federal state*. By drop-down box, you can choose between the key figures *Deaths* or *Deaths per 100,000 inhabitants*.

In [9]:
#hide

# Load deaths by state
data_in = pd.read_excel(io = "https://github.com/joatom/blog-resources/blob/main/mortality_germany/sonderauswertung-sterbefaelle.xlsx?raw=true", 
                        sheet_name = 'BL_2016_2021_Monate_AG_Ins', 
                        header = 8, 
                        index_col = 0,
                        engine='openpyxl')
data_in = data_in.drop(['Unnamed: 16'], axis = 1)

data_in.columns = ['year', 'state', 'age'] + MONTHS

# drop total
data_in = data_in[data_in.age != 'Insgesamt'].reset_index(drop=True)

# clean age column
data_in['age'] = data_in['age'].str.replace(' u. mehr', '+')
# define upper bound of age range
data_in['age_to'] = data_in['age'].map(AGE_MAP)
# map state abbreviations
data_in['state_abbrv'] = data_in['state'].map(STATE_MAP)

# Load population and population density by state

data_pop = pd.read_excel(io = "https://github.com/joatom/blog-resources/blob/main/mortality_germany/02-bundeslaender.xlsx?raw=true", 
                         sheet_name = 'Bundesländer_mit_Hauptstädten', 
                         header = 4, 
                         engine='openpyxl')
data_pop = data_pop[data_pop['Unnamed: 0'].notna()][['Unnamed: 0', 'Unnamed: 2', 'insgesamt', 'je km2']]
data_pop.columns = ['state', 'sq_km', 'population', 'pop_density']
data_pop['state'] = data_pop['state'].str.slice(start=4)

data_pop = data_pop[data_pop['state'].isin(list(STATE_MAP.keys()))].reset_index(drop = True)


# merege deaths data and popuilation data

data_in = data_in.merge(data_pop, on = 'state')

#data_in.head()

In [10]:
#hide

# unpivot by months
data = data_in.melt(id_vars=['year', 'state_abbrv', 'age', 'age_to', 'pop_density', 'population'] , value_vars = MONTHS).dropna()
data.columns = ['year', 'state_abbrv', 'age', 'age_to', 'pop_density', 'population', 'month', 'deaths']

# drop January and February 2016 to get four full years
data = data[(data['year']>2016) | (data['month'].isin(MONTHS[2:]))]

# mark covid period (2020/Mar - 2021/Feb) and non-covid period (2016/Mar - 2020/Feb)
data['period'] = '2020/Mar - 2021/Feb'
data.loc[(data.year < 2020) | ((data.year == 2020) & (data.month.isin(['Jan', 'Feb']))), 'period'] = '2016/Mar - 2020/Feb'

# aggregate data
data = data.groupby(['state_abbrv', 'age', 'period'], as_index=False)['state_abbrv', 'age', 'period', 'pop_density', 'population', 'deaths'].mean()

# define new measures 
data['deaths_per_pop'] = data['deaths'] / data['population'] * 100000

# unpivot by measures 
data_m = data.melt(id_vars=['state_abbrv', 'age', 'period'] , value_vars = ['deaths', 'deaths_per_pop']).dropna()
data_m.columns = ['state_abbrv', 'age', 'period', 'measure', 'val']

#data_m.head()

In [11]:
#hide_input

# allow more then 5000 data entries
#alt.data_transformers.enable('json')
#alt.data_transformers.enable('default', max_rows=6000)

# prepare dropdown
measures = ['deaths', 'deaths_per_pop']
measure_dropdown = alt.binding_select(options=measures, labels = T_STATE_MEASURES, name=T_MEASURE + ': ')
measure_select = alt.selection_single(fields=['measure'], bind=measure_dropdown, name='measure')

base = alt.Chart(data_m).transform_calculate(
    color_age = 'datum.period == "2016/Mar - 2020/Feb" ? (datum.age == "0-65" ? "#106ba3" : "#48aff0") : (datum.age == "0-65" ? "#bf7326" : "#f57c00")'
)

# plot chart
bar_chart = base.mark_bar().encode(
    x= alt.X('val', title = T_DEATHS_MONTHLY),
    y = alt.Y('period', title = ''),
    color = alt.Color('color_age:N', scale=None), #alt.Color('age', legend=alt.Legend(title=T_AGE)),
    row = alt.Row('state_abbrv', header = alt.Header(title = ' ')),
    order = alt.Order(
      'state_abbrv',
      sort ='ascending'
    ),
    tooltip = [alt.Tooltip('state_abbrv', title = T_STATE),
               alt.Tooltip('age', title = T_AGE_GRP),
               alt.Tooltip('val', title = T_DEATHS_MONTHLY, format = '.0f')
              ]
).add_selection(
    measure_select
).transform_filter(
    measure_select  
).properties(width=800, height=50, title = f'{T_DEATHS_BY_STATE} {T_AND} {T_AGE_GRP}')


legend_df =  pd.DataFrame({'age_group' : ['0-65',"0-65 ", '65+','65+ '],
                           'color': [ "#106ba3","#bf7326","#48aff0","#f57c00"],
                          'period': ['2016/Mar - 2020/Feb (0-65)', '2020/Mar - 2021/Feb (0-65)', '2016/Mar - 2020/Feb (65+)', '2020/Mar - 2021/Feb (65+)']})

#https://medium.com/dataexplorations/focus-generating-an-interactive-legend-in-altair-9a92b5714c55
legend = alt.Chart(legend_df
).mark_rect(#filled=True, size=200
).transform_calculate(
    color_legend = 'datum.color'
).encode(
    y=alt.Y('period:O', axis=alt.Axis(orient='right'), title = ' '),
    color=alt.Color('color_legend:N', scale=None),
    order = alt.Order(
      'period:O',
      sort ='ascending'
    )
)

bar_chart | legend

alt.HConcatChart(...)

## Observation
North Rhine-Westphalia (NW) has the largest number of deaths, since it is the state with highest population. In each federal state, there are increases in death rates in the Covid19 period. However, only minimal increases in Hesse (HE) and Bavaria (BY) can be seen for the age groups under the age of 65. In the other federal states, there is no noticeable increase in this age group.

There are smaller variations looking at the number of deaths per 100.000 inhabitants in the age group below 65. The increase in deaths in North Rhine-Westphalia (NW) is slightly lower than in Bavaria (BY) and roughly as high as in Baden-Württemberg (BW). Saxony (SN) and Brandenburg (BB) have the strongest increases.

> Note: Only *deaths* are evaluated. No other aspects (such as demographic aspects) are taken into account.

# Deaths by federal state and population density
The last diagram refers to the *population density* of the *federal states* and the amount of *deaths per 100,000 inhabitants*.

In [12]:
#hide

data_dpp = data.groupby(['state_abbrv', 'period'], as_index=False)['state_abbrv', 'period', 'pop_density','deaths', 'deaths_per_pop'].mean()

data_dpp['deaths_per_pop'] = data_dpp['deaths_per_pop']

data_dpp = data_dpp.pivot(index=['state_abbrv', 'pop_density'], columns = ['period'], values=['deaths', 'deaths_per_pop'])

data_dpp.columns = ['deaths_pre_covid_period', 
                    'deaths_covid_period',
                    'deaths_per_pop_pre_covid_period', 
                    'deaths_per_pop_covid_period']

data_dpp = data_dpp.reset_index()

data_dpp['incr_mortality'] = (data_dpp.deaths_covid_period)/(data_dpp.deaths_pre_covid_period)-1

#data_dpp

In [13]:
#hide_input

scl_min = data_dpp['deaths_per_pop_pre_covid_period'].min() *0.9
scl_max = data_dpp['deaths_per_pop_covid_period'].max() * 1.1

c_tooltip = [alt.Tooltip('state_abbrv', title = T_STATE),
             alt.Tooltip('incr_mortality', title = T_MORT_INC, format='.2%'),
             alt.Tooltip('deaths_pre_covid_period', title = T_DEATHS_BC, format='.0f'),
             alt.Tooltip('deaths_per_pop_pre_covid_period', title = T_PER_HTH, format='.0f'),
             alt.Tooltip('deaths_covid_period', title = T_DEATHS_C, format='.0f'),
             alt.Tooltip('deaths_per_pop_covid_period', title = ' '+T_PER_HTH, format='.0f')
            ]

base = alt.Chart(data_dpp).encode(
    y=alt.Y('pop_density:Q', title = T_DENSITY),
    color = alt.Color('state_abbrv', legend=alt.Legend(title=T_STATE)),
    tooltip = c_tooltip)


c1 = base.mark_point().encode(
    x=alt.X('deaths_per_pop_pre_covid_period:Q', 
            title = T_DEATHS_HTH, 
            scale=alt.Scale(domain=(scl_min, scl_max))),
    size=alt.Size('deaths_pre_covid_period:Q', legend=alt.Legend(title=T_DEATHS_MONTHLY))
).interactive(bind_x = False)

c2 = base.mark_circle().encode(
    x=alt.X('deaths_per_pop_covid_period:Q', 
            title = T_DEATHS_HTH, 
            scale=alt.Scale(domain=(scl_min, scl_max))),
    size= alt.Size('deaths_covid_period:Q', legend=alt.Legend(title=T_DEATHS_MONTHLY))
).interactive(bind_x = False)

diffbar = base.mark_errorbar().encode(
    x= alt.X('deaths_per_pop_pre_covid_period:Q', 
            title = T_DEATHS_HTH),
    x2= alt.X2("deaths_per_pop_covid_period:Q", 
            title = T_DEATHS_HTH)
)

(alt.layer( c1 + c2 + diffbar)
    .properties(width=800, height=500, title = f'{T_DEATHS_BY_STATE} {T_AND} {T_DENSITY2}')
    .configure_title(fontSize=15, offset=15, orient='top', anchor='middle')
)

alt.LayerChart(...)

## Observation

The city-states (Hamburg (HH), Bremen (HB) and Berlin (BE)) show a moderate increase in deaths despite their high population density. Schleswig-Holstein (SH) recorded the lowest increase.

> Note: Only deaths are evaluated. No other aspects (such as demographic aspects) are taken into account.

# Technical conclusion
With Altair you can easily setup interactive charts and integrate them into notebooks. Their website contains a variety of examples [AA1]. Some questions are also answered in their Github Issue tracker [AA2].

I spent some time debugging JavaScript when I used the bindings incorrectly. However, there was finally a solution to every problem [AA2] or workarounds such as for the legend in the bar diagram [AG18]. As expected, a bit of practice is neccessary to get used to a new framework.

The integration [FP20] in this blog works perfectly. I will certainly use the framework elsewhere.

# References
## Data sources
The data used here are from the "Statistisches Bundesamt" (Federal Statistical Office) and are subject to the license "dl-de/by-2-0". The license text can be found at www.govdata.de/dl-de/by-2-0. The data were modified exclusively within this notebook by executing the specified program code for the purpose of analysis.
- [SB21] Statistisches Bundesamt (Destatis), 2021 (published 2012/03/30), *Sterbefälle - Fallzahlen nach Tagen, Wochen, Monaten, Altersgruppen, Geschlecht und Bundesländern für Deutschland 2016 - 2021*, visited 2021/04/03, <https://www.destatis.de/DE/Themen/Gesellschaft-Umwelt/Bevoelkerung/Sterbefaelle-Lebenserwartung/Tabellen/sonderauswertung-sterbefaelle.xlsx?__blob=publicationFile>
- [SB20] Statistisches Bundesamt (Destatis), 2020 (published 2020/09/02), *Bundesländer mit Hauptstädten nach Fläche, Bevölkerung und Bevölkerungsdichte am 31.12.2019*, visited 2021/04/03, <https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/02-bundeslaender.xlsx?__blob=publicationFile>

## Other references
A lot of the coding is derived from various examples of the Altair homepage and from great examples in the coresponding Github Issue tracker answered by https://github.com/jakevdp.
- [AA1] https://altair-viz.github.io/gallery/index.html
- [AA2] https://github.com/altair-viz/altair/issues/
- [AG18] A. Gordon, 2018 (published 2018/10/06), *Focus: generating an interactive legend in Altair*, visited 2021/04/05, <https://medium.com/dataexplorations/focus-generating-an-interactive-legend-in-altair-9a92b5714c55>
- [FP20] fastpages.fast.ai, 2020 (published 2020/02/20), *Fastpages Notebook Blog Post*, visited 2021/04/05, <https://fastpages.fast.ai/jupyter/2020/02/20/test.html> 